# Understanding the relationship between COVID-19 and Opioid overdose deaths in the US

Import statements and data you need

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
%matplotlib inline

## Data Cleaning

In [2]:
# dataframe for first csv

# Path to the data - select the path that works for you 
file_path = '../data-mining-semester-project/data/Census race.csv'

# Load the data into a DataFrame 
dfCensus = pd.read_csv(file_path)

dfCensus = dfCensus.dropna()
dfCensus= dfCensus.replace('!!',' ', regex=True)

# replace column headers with first row (more useful imo)
header = dfCensus.iloc[0]
dfCensus = dfCensus[1:]
dfCensus = dfCensus.rename(columns = header)

# dfCensus.info()
dfCensus.head()

,id,Geographic Area Name,Total:,Total: Population of one race:,Total: Population of one race: White alone,Total: Population of one race: Black or African American alone,Total: Population of one race: American Indian and Alaska Native alone,Total: Population of one race: Asian alone,Total: Population of one race: Native Hawaiian and Other Pacific Islander alone,Total: Population of one race: Some Other Race alone,...,Total: Population of two or more races: Population of four races: American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race,Total: Population of two or more races: Population of five races:,Total: Population of two or more races: Population of five races: White; Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander,Total: Population of two or more races: Population of five races: White; Black or African American; American Indian and Alaska Native; Asian; Some Other Race,Total: Population of two or more races: Population of five races: White; Black or African American; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander; Some Other Race,Total: Population of two or more races: Population of five races: White; Black or African American; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race,Total: Population of two or more races: Population of five races: White; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race,Total: Population of two or more races: Population of five races: Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race,Total: Population of two or more races: Population of six races:,Total: Population of two or more races: Population of six races: White; Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race
1,0500000US01001,"Autauga County, Alabama",58805,55648,42160,11445,217,881,35,910,...,0,2,2,0,0,0,0,0,0,0
2,0500000US01003,"Baldwin County, Alabama",231767,216743,189399,18217,1582,2067,143,5335,...,0,0,0,0,0,0,0,0,0,0
3,0500000US01005,"Barbour County, Alabama",25223,24523,11317,11933,116,117,1,1039,...,0,2,2,0,0,0,0,0,0,0
4,0500000US01007,"Bibb County, Alabama",22293,21534,16555,4413,60,32,9,465,...,0,0,0,0,0,0,0,0,0,0
5,0500000US01009,"Blount County, Alabama",59134,55478,50663,845,337,178,24,3431,...,0,1,0,0,1,0,0,0,0,0


In [3]:
# dataframe for second csv file 
file_path2 = '../data-mining-semester-project/data/Unemployment.csv'

# Load the data into a DataFrame 
dfUnemployment = pd.read_csv(file_path2)
dfUnemployment_all = dfUnemployment.copy()
dfUnemployment = dfUnemployment.iloc[:,[1,2,86,87,88,89]]

# dfUnemployment = dfUnemployment.dropna()
# # getting rid of values outside of 2015-2020
# dfUnemployment = dfUnemployment.drop(dfUnemployment[dfUnemployment.loc["State"]=="AK"])

# drop anything not in Alabama, Arkansas, Delaware, the District of Columbia, Florida, Georgia, Kentucky, Louisiana, Maryland, Mississippi, North Carolina, Oklahoma, South Carolina, Tennessee, Texas, Virginia, and West Virginia.

dfUnemployment.head()

,State,Area_name,Civilian_labor_force_2020,Employed_2020,Unemployed_2020,Unemployment_rate_2020
0,US,United States,"160,611,064","147,677,360","12,933,704",8.1
1,AL,Alabama,"2,230,118","2,099,062","131,056",5.9
2,AL,"Autauga County, AL","25,838","24,576","1,262",4.9
3,AL,"Baldwin County, AL","96,763","91,338","5,425",5.6
4,AL,"Barbour County, AL","8,587","7,982",605,7.0


In [4]:
# dataframe for fourth csv file 

# Underlying Cause of Death, 1999-2019.csv
file_path4 = '../data-mining-semester-project/data/Underlying Cause of Death, 1999-2019.csv'

# drop anything not in Alabama, Arkansas, Delaware, the District of Columbia, Florida, Georgia, Kentucky, Louisiana, Maryland, Mississippi, North Carolina, Oklahoma, South Carolina, Tennessee, Texas, Virginia, and West Virginia.
# reduce time to 2015 to 2020 for this dataset 
# Load the data into a DataFrame 
dfCDeath = pd.read_csv(file_path4)
dfCDeath.head()
# dfCDeath.info()

,County,County Code,Drug/Alcohol Induced,Drug/Alcohol Induced Code,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths,Population,Crude Rate,Crude Rate Lower 95% Confidence Interval,Crude Rate Upper 95% Confidence Interval,Crude Rate Standard Error,Age Adjusted Rate,Age Adjusted Rate Lower 95% Confidence Interval,Age Adjusted Rate Upper 95% Confidence Interval,% of Total Deaths
0,"Autauga County, AL",1001,All other non-drug and non-alcohol causes,O,All other non-drug and non-alcohol causes,O9,536.0,55869.0,959.4,878.2,1040.6,41.4,812.2,742.7,881.7,0.0%
1,"Baldwin County, AL",1003,Drug-induced causes,D,Drug poisonings (overdose) Unintentional (X40-...,D1,26.0,223234.0,11.6,7.6,17.1,2.3,13,8.3,19.3,0.0%
2,"Baldwin County, AL",1003,Alcohol-induced causes,A,All other alcohol-induced causes,A9,27.0,223234.0,12.1,8.0,17.6,2.3,10.8,6.9,16.0,0.0%
3,"Baldwin County, AL",1003,All other non-drug and non-alcohol causes,O,All other non-drug and non-alcohol causes,O9,2222.0,223234.0,995.4,954.0,1036.8,21.1,721.3,690.2,752.3,0.1%
4,"Barbour County, AL",1005,All other non-drug and non-alcohol causes,O,All other non-drug and non-alcohol causes,O9,330.0,24686.0,1336.8,1192.6,1481.0,73.6,1021.4,907.8,1135.0,0.0%


In [5]:
# state death data covid19
file_path5 = '../data-mining-semester-project/data/statesCOVID.csv'

dfStateC19Death = pd.read_csv(file_path5)
dfStateC19Death.head()

dfStateC19Death = dfStateC19Death[(dfStateC19Death['date'] >= '2020-01-01') & (dfStateC19Death['date'] <= '2020-12-31')]
dfStateC19Death['date'] = pd.to_datetime(dfStateC19Death['date'])
dfStateC19Death.head()

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0


In [6]:
#county death data covid19
file_path6 = '../data-mining-semester-project/data/us-counties.csv'
# drop anything not a southern state 
dfCountyC19Death = pd.read_csv(file_path6)
dfCountyC19Death.head()

,date,geoid,county,state,cases,cases_avg,cases_avg_per_100k,deaths,deaths_avg,deaths_avg_per_100k
0,2020-01-21,USA-53061,Snohomish,Washington,1.0,0.14,0.02,0.0,0.0,0.0
1,2020-01-22,USA-53061,Snohomish,Washington,0.0,0.14,0.02,0.0,0.0,0.0
2,2020-01-23,USA-53061,Snohomish,Washington,0.0,0.14,0.02,0.0,0.0,0.0
3,2020-01-24,USA-53061,Snohomish,Washington,0.0,0.14,0.02,0.0,0.0,0.0
4,2020-01-24,USA-17031,Cook,Illinois,1.0,0.14,0.00,0.0,0.0,0.0


In [7]:
# VSRR_Provisional_Drug_Overdose_Death_Counts.csv
file_path3 = '../data-mining-semester-project/data/VSRR_Provisional_Drug_Overdose_Death_Counts.csv'

# Load the data into a DataFrame 
dfODeath = pd.read_csv(file_path3)
# dfODeath = dfODeath.dropna()
dfODeath = dfODeath.drop(["Footnote"],axis=1)
dfODeath = dfODeath.drop(["Footnote Symbol"],axis=1)
# dfODeath = dfODeath.drop(["State Name"],axis=1)
# dfODeath = dfODeath.drop([dfODeath["State"]=="AK"],1)
# dfODeath

In [8]:
# need to drop anything that is not Alabama, Arkansas, Delaware, the District of Columbia, Florida, Georgia, Kentucky, Louisiana, Maryland, Mississippi, North Carolina, Oklahoma, South Carolina, Tennessee, Texas, Virginia, and West Virginia.
# dfODeath.head()

# death data for type of drug 
# VSRR_Provisional_Drug_Overdose_Death_Counts.csv
file_path4 = '../data-mining-semester-project/data/VSRR_Provisional_Drug_Overdose_Death_Counts-3.csv'

# drop anything not in a southern state to start, retain data from 2015 to end of 2020

# Load the data into a DataFrame 
dfTypeDrug = pd.read_csv(file_path4)
dfTypeDrug = dfTypeDrug.drop(["Period"],axis=1)
dfTypeDrug = dfTypeDrug.drop(["Footnote"],axis=1)
dfTypeDrug = dfTypeDrug.drop(["Footnote Symbol"],axis=1)

# changing Month and Year columns to "date" column for easier plotting
dfTypeDrug['Date'] = dfTypeDrug['Month'].map(str)+ '-' +dfTypeDrug['Year'].map(str)
dfTypeDrug['Date'] = pd.to_datetime(dfTypeDrug['Date']) #, format='%m-%Y') #.dt.strftime('%m-%Y')
dfTypeDrug = dfTypeDrug.drop(["Year"],axis=1)
dfTypeDrug = dfTypeDrug.drop(["Month"],axis=1)

# for now, just using "Opioids (T40.0-T40.4,T40.6)" as the data 
dfTypeDrug = dfTypeDrug[(dfTypeDrug['Indicator'] == "Opioids (T40.0-T40.4,T40.6)")]

dfTypeDrug = dfTypeDrug[(dfTypeDrug['State'] != "US")]

dfTypeDrug['Data Value'] = dfTypeDrug['Data Value'].str.replace(',', '')
dfTypeDrug = dfTypeDrug.astype({'Data Value': float}) 

df_18_20_Opioid = dfTypeDrug[(dfTypeDrug['Date'] >= '2018-01-01') & (dfTypeDrug['Date'] <= '2020-12-31')].copy()
df_18_20_Opioid['Total Deaths per Month'] = df_18_20_Opioid.groupby([pd.Grouper(key='Date', freq='1M')])['Data Value'].transform('sum')
df_18_20_Opioid = df_18_20_Opioid[['Date', 'Total Deaths per Month']]
df_18_20_Opioid = df_18_20_Opioid.drop_duplicates()
df_18_20_Opioid = df_18_20_Opioid.sort_values('Date')
df_18_20_Opioid.head()

,Date,Total Deaths per Month
491,2018-01-01,27722.0
478,2018-02-01,29541.0
526,2018-03-01,29689.0
437,2018-04-01,28506.0
528,2018-05-01,28385.0


## Plotting Functions

In [9]:
def scatterplot(x_data, y_data, xlabel, ylabel):
    plt.scatter(x_data, y_data)
#     locator = matplotlib.dates.YearLocator()
#     plt.gca().xaxis.set_major_locator(locator)
#     plt.gcf().autofmt_xdate()
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
#     plt.title(title)
    plt.show()

def timeplot(x_data, y_data, xlabel, ylabel):
    plt.plot(x_data, y_data)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
#     locator = matplotlib.dates.YearLocator()
#     plt.gca().xaxis.set_major_locator(locator)
    plt.gcf().autofmt_xdate()
#     plt.title(title)        
#     plt.legend()
    plt.show()
    
def descriptivestatistics(df, col_header):
    n = df[col_header].count()
    mean = df[col_header].mean()
    median = df[col_header].quantile(.5)
    Q1 = df[col_header].quantile(.25)
    Q3 = df[col_header].quantile(.75)
    IQR = Q3 - Q1
    stdev = df[col_header].std()
    skewness = df[col_header].skew()
    kurtosis = df[col_header].kurt()
    print(("Data for %s\nn \t= %f\nmean \t= %f\nmedian \t= %f\nQ1 \t= %f\nQ3 \t= %f\nIQR \t= %f\nstdev \t= %f\nskewness = %f\nkurtosis = %f\n") 
              % (col_header, n, mean, median, Q1, Q3, IQR, stdev, skewness, kurtosis))

In [25]:
# opioid data
# filter out non-opioid statistics for VSRR
# what do we want to use as our y-axis:
    # we could use "Number of Drug Overdose Deaths"
    # or use just "Opioids (T40.0-T40.4,T40.6)" which covers all the diff types of opioids <-- gonna lean towards this
# do we want to use predicted values for data over raw value because this is provisional and they have that available?
# ^ for now i'm not going to because it's not for sure but it shouldn't be hard to just replace the data value
# also large portions of missing data, I assume we can use regression (used mean instead) to make predictions on what that data may have been (or we could just drop those values)
dfOpioid2020 = dfTypeDrug[(dfTypeDrug['Date'] >= '2020-01-01') & (dfTypeDrug['Date'] <= '2020-12-31')]
dfOpioid2020 = dfOpioid2020[['State Name', 'Date', 'Data Value']]
dfOpioid2020 = dfOpioid2020.sort_values('State Name')
dfOpioid2020['Total Deaths'] = (dfOpioid2020.groupby([pd.Grouper(key='State Name')])['Data Value'].transform('sum'))
dfOpioid2020['Total Available Months'] = (dfOpioid2020.groupby([pd.Grouper(key='State Name')])['Data Value'].transform('count'))
TotalDeaths = dfOpioid2020['Total Deaths'].to_numpy()
TotalMonths = dfOpioid2020['Total Available Months'].to_numpy()
NewTotalDeaths = np.array(TotalDeaths/TotalMonths * 12)
NewTotalDeaths = pd.DataFrame(NewTotalDeaths)
dfOpioid2020.insert(2, "Total Opioid Deaths 2020", NewTotalDeaths)
dfOpioid2020byMonth = dfOpioid2020.copy()
dfOpioid2020 = dfOpioid2020.iloc[:,[0,2]]
dfOpioid2020 = dfOpioid2020.drop_duplicates()
dfOpioid2020byMonth = dfOpioid2020byMonth.iloc[:,[0,1,3]]
dfOpioid2020.head()
# dfOpioid2020byMonth

,State Name,Total Opioid Deaths 2020
721,Alaska,1035.0
2157,Arizona,20167.0
3057,California,54222.0
4005,Colorado,10130.0
4891,Connecticut,14577.0


In [11]:
# COVID data
# filter out states we don't care about
# filter just 2020 timeframe
dfCOVID2020 = dfStateC19Death.copy()
dfCOVID2020 = dfCOVID2020.drop(["fips"],axis=1)

# total by month for each state
dfCOVID2020 = dfCOVID2020.sort_values('date')
dfCOVID2020 = dfCOVID2020.sort_values('state')

dfCOVID2020['Total COVID Cases 2020'] = (dfCOVID2020.groupby([pd.Grouper(key='state')])['cases'].transform('max'))
dfCOVID2020['Total COVID Deaths 2020'] = (dfCOVID2020.groupby([pd.Grouper(key='state')])['deaths'].transform('max'))
dfCOVID2020 = dfCOVID2020.iloc[:,[1,4,5]]
dfCOVID2020 = dfCOVID2020.drop_duplicates()
dfCOVID2020.head()

,state,Total COVID Cases 2020,Total COVID Deaths 2020
12609,Alabama,361226,4827
1720,Alaska,46740,198
14261,Arizona,523829,8879
12887,Arkansas,225138,3676
1448,California,2307860,25965


In [12]:
# drop the states we don't have data for (at least for now)
# Alabama, Arkansas, Florida, Idaho, Louisiana, Minnesota, North Dakota, Nebraska, Pennsylvania
# as well as drop American Samoa, Guam, Northern Mariana Islands, Virgin Islands, and Puerto Rico
# and NYC for opioid set

dfCOVID2020 = dfCOVID2020[(dfCOVID2020['state'] != "Alabama") &
                          (dfCOVID2020['state'] != "Arkansas") &
                          (dfCOVID2020['state'] != "Florida") &
                          (dfCOVID2020['state'] != "Idaho") &
                          (dfCOVID2020['state'] != "Louisiana") &
                          (dfCOVID2020['state'] != "Minnesota") &
                          (dfCOVID2020['state'] != "North Dakota") &
                          (dfCOVID2020['state'] != "Nebraska") &
                          (dfCOVID2020['state'] != "Pennsylvania") &
                          (dfCOVID2020['state'] != "American Samoa") &
                          (dfCOVID2020['state'] != "Guam") &
                          (dfCOVID2020['state'] != "Northern Mariana Islands") &
                          (dfCOVID2020['state'] != "Virgin Islands") & 
                          (dfCOVID2020['state'] != "Puerto Rico")]
dfOpioid2020 = dfOpioid2020[(dfOpioid2020['State Name'] != "New York City")]
dfOpioid2020byMonth = dfOpioid2020byMonth[(dfOpioid2020byMonth['State Name'] != "New York City")]

stateswithavailabledata = dfCOVID2020['state'].unique()
# dfCOVID2020.info()
# dfOpioid2020.info()
# dfOpioid2020byMonth

In [13]:
# make it per capita...
dfCensus["State"] = dfCensus["Geographic Area Name"].str.replace('(.+.+.+.+ \w+, )','',regex=True)
dfCensus["State"] = dfCensus["State"].str.replace('(.+ \w+, )','',regex=True)
dfCensus2020 = dfCensus
# i gave up on trying to make the col headers nice
dfCensus2020 = dfCensus2020.astype({'  Total:': int}) 
dfCensus2020['Total State Population'] = (dfCensus2020.groupby([pd.Grouper(key='State')])['  Total:'].transform('sum'))
dfCensus2020_pop = dfCensus2020[['State', 'Total State Population']]
dfCensus2020_pop = dfCensus2020_pop.drop_duplicates()
dfCensus2020_pop2 = dfCensus2020_pop.copy()
dflist = []
for x in stateswithavailabledata:
    dflist.append(dfCensus2020_pop[(dfCensus2020_pop['State'] == x)])

temp = pd.DataFrame()

for x in dflist:
    temp = temp.append(x)

dfCensus2020_pop = temp.drop_duplicates()
dfCensus2020_pop.head()

,State,Total State Population
68,Alaska,733391
98,Arizona,7151502
188,California,39538223
246,Colorado,5773714
310,Connecticut,3605944


In [14]:
stateswithavailabledata
dfUnemployment['Civilian_labor_force_2020'] = dfUnemployment['Civilian_labor_force_2020'].str.replace(',', '')
dfUnemployment['Unemployed_2020'] = dfUnemployment['Unemployed_2020'].str.replace(',', '')

dflist = []
for x in stateswithavailabledata:
    dflist.append(dfUnemployment[(dfUnemployment['Area_name'] == x)])

dfUnemployment2020 = pd.DataFrame()

for x in dflist:
    dfUnemployment2020 = dfUnemployment2020.append(x)

dfUnemployment2020 = dfUnemployment2020.drop_duplicates()
dfUnemployment2020.head()

,State,Area_name,Civilian_labor_force_2020,Employed_2020,Unemployed_2020,Unemployment_rate_2020
69,AK,Alaska,347414,"320,219",27195,7.8
102,AZ,Arizona,3570220,"3,288,150",282070,7.9
194,CA,California,18821167,"16,913,078",1908089,10.1
253,CO,Colorado,3122237,"2,895,473",226764,7.3
318,CT,Connecticut,1872631,"1,724,621",148010,7.9


In [15]:
dfCOVID2020byMonth = dfStateC19Death.copy()
dfCOVID2020byMonth = dfCOVID2020byMonth.drop(["fips"],axis=1)

dflist = []
for x in stateswithavailabledata:
    dflist.append(dfCOVID2020byMonth[(dfCOVID2020byMonth['state'] == x)].copy())

temp = pd.DataFrame()

for x in dflist:
    x['Monthly COVID Cases 2020'] = (x.groupby([pd.Grouper(key='date', freq='1M')])['cases'].transform('max'))
    x['Monthly COVID Deaths 2020'] = (x.groupby([pd.Grouper(key='date', freq='1M')])['deaths'].transform('max'))
    temp = temp.append(x)

dfCOVID2020byMonth = temp
dfCOVID2020byMonth = dfCOVID2020byMonth.drop(["cases"],axis=1)
dfCOVID2020byMonth = dfCOVID2020byMonth.drop(["deaths"],axis=1)

dfCOVID2020byMonth['date'] = dfCOVID2020byMonth['date'].dt.month
dfCOVID2020byMonth = dfCOVID2020byMonth.drop_duplicates()
dfCOVID2020byMonth = dfCOVID2020byMonth.astype({'date': int})

# change everthing to a numpy array to iterate through and extract the monthly total 
# instead of the cumulative total
datearray = dfCOVID2020byMonth['date'].to_numpy()
statearray = dfCOVID2020byMonth['state'].to_numpy()
casesarray = dfCOVID2020byMonth['Monthly COVID Cases 2020'].to_numpy()
deatharray = dfCOVID2020byMonth['Monthly COVID Deaths 2020'].to_numpy()

for i in range(len(datearray)):
    if ((datearray[i] > 1) & (statearray[i] == statearray[i-1])):
        deatharray[i] -= deatharray[i-1]
        casesarray[i] -= casesarray[i-1]

# fill in df if the state does not have data for a month (likely January/February/March)
# this solution really sucks computationally but I guess it works
dates = dfCOVID2020byMonth['date'].unique()
dates = np.sort(dates)

dflist = []
for x in stateswithavailabledata:
    dflist.append(dfCOVID2020byMonth[(dfCOVID2020byMonth['state'] == x)].copy())

emptyrows = []
    
for x in dflist:
    x_datearray = x['date'].to_numpy()
    x_statearray = x['state'].to_numpy()
    for i, date in enumerate(dates):
        if(date not in x_datearray):
            emptyrows.append([date, x_statearray[i], 0, 0])

headers = dfCOVID2020byMonth.iloc[0]
emptyrows = pd.DataFrame(emptyrows)
emptyrows = emptyrows.rename(columns = {0:"date", 1:"state", 2:"Monthly COVID Cases 2020", 3:"Monthly COVID Deaths 2020"})
dfCOVID2020byMonth = dfCOVID2020byMonth.append(emptyrows, ignore_index = True)
dfCOVID2020byMonth = dfCOVID2020byMonth.sort_values('state')
dfCOVID2020byMonth = dfCOVID2020byMonth.astype({'date': str})
datearray = dfCOVID2020byMonth['date'].to_numpy()
for i, x in enumerate(datearray):
    x = x.replace(x, x + '-01-2020')
    datearray[i] = x

dfCOVID2020byMonth['date'] = pd.to_datetime(dfCOVID2020byMonth['date'])
dfCOVID2020byMonth
dfCOVIDbyMonth = dfCOVID2020byMonth.copy()
dfCOVIDbyMonth['Total Monthly COVID Cases'] = dfCOVIDbyMonth.groupby([pd.Grouper(key='date', freq='1M')])['Monthly COVID Cases 2020'].transform('sum')
dfCOVIDbyMonth['Total Monthly COVID Deaths'] = dfCOVIDbyMonth.groupby([pd.Grouper(key='date', freq='1M')])['Monthly COVID Deaths 2020'].transform('sum')
dfCOVIDbyMonth = dfCOVIDbyMonth[['date','Total Monthly COVID Cases', 'Total Monthly COVID Deaths']]
dfCOVIDbyMonth = dfCOVIDbyMonth.drop_duplicates()
dfCOVIDbyMonth = dfCOVIDbyMonth.sort_values('date')
dfCOVIDbyMonth

,date,Total Monthly COVID Cases,Total Monthly COVID Deaths
433,2020-01-01,7,0
434,2020-02-01,50,1
0,2020-03-01,167982,3846
1,2020-04-01,772191,52610
2,2020-05-01,780104,38686
3,2020-06-01,1438486,72075
4,2020-07-01,2172796,59195
5,2020-08-01,2583148,94188
6,2020-09-01,3140339,76678
7,2020-10-01,4160938,112131


In [16]:
# fill in df if the state does not have data for a month (likely January/February/March)
# this solution really sucks computationally but I guess it works

dfOpioid2020byMonth['Average Deaths'] = dfOpioid2020byMonth.groupby([pd.Grouper(key='State Name')])['Data Value'].transform('mean')
dflist = []
for x in stateswithavailabledata:
    dflist.append(dfOpioid2020byMonth[(dfOpioid2020byMonth['State Name'] == x)].copy())
    
emptyrows = []
    
for x in dflist:
    x_datearray = x['Date'].to_numpy(dtype='datetime64[D]')
    x_statearray = x['State Name'].to_numpy()
    x_data = x['Data Value'].to_numpy()
    x_avgdata = x['Average Deaths'].to_numpy()
    for i, data in enumerate(x_data):
        if np.isnan(x_data[i]):
            emptyrows.append([x_statearray[i], x_datearray[i], x_avgdata[i], x_avgdata[i]])

headers = dfOpioid2020byMonth.iloc[0]
emptyrows = pd.DataFrame(emptyrows)
emptyrows = emptyrows.rename(columns = {0:"State Name", 1:"Date", 2:"Data Value", 3:"Average Deaths"})

# dfOpioid2020byMonth
dfOpioid2020byMonth = dfOpioid2020byMonth.append(emptyrows, ignore_index = True)
dfOpioid2020byMonth = dfOpioid2020byMonth.sort_values('State Name')
dfOpioid2020byMonth = dfOpioid2020byMonth.dropna()
dfOpioid2020byMonth = dfOpioid2020byMonth.astype({'Date': str})
dfOpioid2020byMonth['Date'] = pd.to_datetime(dfOpioid2020byMonth['Date'])
dfOpioid2020byMonth

/tmp/ipykernel_130/1915076331.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfOpioid2020byMonth['Average Deaths'] = dfOpioid2020byMonth.groupby([pd.Grouper(key='State Name')])['Data Value'].transform('mean')


,State Name,Date,Data Value,Average Deaths
0,Alaska,2020-04-01,74.0,86.250000
11,Alaska,2020-07-01,93.0,86.250000
10,Alaska,2020-08-01,98.0,86.250000
9,Alaska,2020-12-01,102.0,86.250000
7,Alaska,2020-01-01,80.0,86.250000
...,...,...,...,...
495,Wyoming,2020-05-01,51.0,53.083333
494,Wyoming,2020-11-01,63.0,53.083333
493,Wyoming,2020-10-01,60.0,53.083333
501,Wyoming,2020-12-01,60.0,53.083333


In [17]:
# Make per-capita df
temp = dfCensus2020_pop.copy()
states = dfCensus2020_pop['State'].to_numpy()
populations = dfCensus2020_pop['Total State Population'].to_numpy()
covidcases = dfCOVID2020['Total COVID Cases 2020'].to_numpy()
coviddeaths = dfCOVID2020['Total COVID Deaths 2020'].to_numpy()
opioiddeaths = dfOpioid2020['Total Opioid Deaths 2020'].to_numpy()

covidcasespercapita = np.array(covidcases/populations)
coviddeathspercapita = np.array(coviddeaths/populations)
opioiddeathspercapita = np.array(opioiddeaths/populations)

rows = states
cols=['C19CasesPerCapita', 'C19DeathsPerCapita', 'OpioidDeathsPerCapita']
df1 = pd.DataFrame(covidcasespercapita, index=rows)
df2 = pd.DataFrame(coviddeathspercapita, index=rows)
df3 = pd.DataFrame(opioiddeathspercapita, index=rows)
data=[df1, df2, df3]
dfpercapita = pd.concat(data,axis=1)
dfpercapita.columns = cols
dfpercapita.head()

,C19CasesPerCapita,C19DeathsPerCapita,OpioidDeathsPerCapita
Alaska,0.063731,0.000270,0.001411
Arizona,0.073247,0.001242,0.002820
California,0.058370,0.000657,0.001371
Colorado,0.058122,0.000845,0.001755
Connecticut,0.051501,0.001663,0.004042


In [47]:
# Missing Opioid data 2020:
# Cluster states by population, total drug od death
# Fill in by getting ratio of Opioid/Total for same cluster states
# Declare those as estimates
# columns of this df should be:
# State - Indicator - Data Value - Population

dfDrugCluster = dfODeath[['State Name', 'Year', 'Month', 'Indicator', 'Data Value']].copy()

# changing Month and Year columns to "date" column for easier plotting
dfDrugCluster['Date'] = dfDrugCluster['Month'].map(str)+ '-' +dfDrugCluster['Year'].map(str)
dfDrugCluster['Date'] = pd.to_datetime(dfDrugCluster['Date'])
dfDrugCluster = dfDrugCluster.drop(["Year"],axis=1)
dfDrugCluster = dfDrugCluster.drop(["Month"],axis=1)
dfDrugCluster = dfDrugCluster.sort_values('State Name')
dfDrugCluster = dfDrugCluster[(dfDrugCluster['Date'] >= '2020-01-01') & (dfDrugCluster['Date'] <= '2020-12-31')]
dfDrugCluster = dfDrugCluster[(dfDrugCluster['Indicator'] == "Opioids (T40.0-T40.4,T40.6)") + (dfDrugCluster['Indicator'] == "Number of Drug Overdose Deaths")]
dfDrugCluster['Data Value'] = dfDrugCluster['Data Value'].str.replace(',', '')
dfDrugCluster = dfDrugCluster.astype({'Data Value': float}) 

dflist = []
for x in dfDrugCluster['State Name'].unique():
    dflist.append(dfDrugCluster[(dfDrugCluster['State Name'] == x)].copy())

temp = pd.DataFrame()
    
for x in dflist:
    x['Total Deaths'] = (x.groupby([pd.Grouper(key='Indicator')])['Data Value'].transform('sum'))
    temp = temp.append(x)

dfDrugCluster = temp.drop(["Date"],axis=1)
dfDrugCluster = dfDrugCluster.drop(["Data Value"],axis=1)
dfDrugCluster = dfDrugCluster.drop_duplicates()
dfDrugCluster2 = dfDrugCluster[(dfDrugCluster['Indicator'] == "Number of Drug Overdose Deaths")]
dfDrugCluster2 = dfDrugCluster2[(dfDrugCluster2['State Name'] != "United States") & (dfDrugCluster2['State Name'] != "New York City")]

dflist = []
for x in dfDrugCluster2['State Name'].unique():
    dflist.append(dfUnemployment[(dfUnemployment['Area_name'] == x)])

dfDrugCluster2 = dfDrugCluster2.set_index('State Name')
dfCensus2020_pop2 = dfCensus2020_pop2[(dfCensus2020_pop2['State'] != "Puerto Rico")]
rows = dfCensus2020_pop2['State'].unique()
dfCensus2020_pop3 = dfCensus2020_pop2.set_index('State')
dfDrugCluster2 = dfDrugCluster2.drop(["Indicator"],axis=1)

dfUnemploymentCluster = pd.DataFrame()

for x in dflist:
    dfUnemploymentCluster = dfUnemploymentCluster.append(x)

dfUnemploymentCluster = dfUnemploymentCluster.drop_duplicates()
dfUnemploymentCluster.head()


a1 = dfDrugCluster2['Total Deaths'].to_numpy()
a2 = dfCensus2020_pop3['Total State Population'].to_numpy()
a3 = dfUnemploymentCluster['Unemployment_rate_2020'].to_numpy()
df1 = pd.DataFrame(a1, index=rows)
df2 = pd.DataFrame(a2, index=rows)
df3 = pd.DataFrame(a3, index=rows)

cols=['Drug OD', 'Population', 'Unemployment Rate']
data=[df1, df2, df3]
dfcluster = pd.concat(data,axis=1)
dfcluster.columns = cols
dfcluster.head()

from sklearn.cluster import KMeans

km = KMeans(
    n_clusters=3, init='random',
    n_init=10, max_iter=3000, 
    tol=1e-04, random_state=1
)
y_km = km.fit_predict(dfcluster)

dfOpioid2020_2 = dfOpioid2020.set_index('State Name').copy()

df4 = pd.DataFrame(y_km, index=rows)
df5 = pd.DataFrame(rows, index=rows)
df6 = pd.DataFrame(dfOpioid2020_2['Total Opioid Deaths 2020'], index=rows)
dfcluster = pd.concat([dfcluster,df4,df5,df6],axis=1)
cols.append('Cluster')
cols.append('State')
cols.append('Opioid OD')
dfcluster.columns = cols

# now that I have the clusters, I need the average proportion of opioid OD/drug OD deaths per cluster 
# once I have that proportion, multiply it by drug OD deaths for an estimate of opioid OD deaths on states with missing data

drugOD = dfcluster['Drug OD'].to_numpy()
opioidOD = dfcluster['Opioid OD'].to_numpy()
opioidOD_rate = np.array(opioidOD/drugOD)

df7 = pd.DataFrame(opioidOD_rate, index=rows)
dfcluster = pd.concat([dfcluster,df7],axis=1)
cols.append('Opioid OD Percentage')
dfcluster.columns = cols
dfcluster
    
dfcluster['Average Opioid OD Percentage'] = dfcluster.groupby([pd.Grouper(key='Cluster')])['Opioid OD Percentage'].transform('mean')
avgrate = dfcluster['Average Opioid OD Percentage'].to_numpy()
opioidOD_estimate = np.array(avgrate*drugOD)
opioidOD_estimate = np.round(opioidOD_estimate)
opioidOD_estimate = opioidOD_estimate.astype(int)

df8 = pd.DataFrame(opioidOD_estimate, index=rows)
dfcluster = pd.concat([dfcluster,df8],axis=1)
cols.append('Average Opioid OD Percentage')
cols.append('Opioid OD Estimate')
dfcluster.columns = cols
dfcluster

,Drug OD,Population,Unemployment Rate,Cluster,State,Opioid OD,Opioid OD Percentage,Average Opioid OD Percentage,Opioid OD Estimate
Alabama,10624.0,5024279,5.9,2,Alabama,NaN,NaN,0.672848,7148
Alaska,1591.0,733391,7.8,2,Alaska,1035.0,0.650534,0.672848,1071
Arizona,28466.0,7151502,7.9,1,Arizona,20167.0,0.708459,0.783713,22309
Arkansas,5325.0,3011524,6.1,2,Arkansas,NaN,NaN,0.672848,3583
California,92810.0,39538223,10.1,0,California,54222.0,0.584226,0.633908,58833
Colorado,16085.0,5773714,7.3,1,Colorado,10130.0,0.629779,0.783713,12606
Connecticut,16012.0,3605944,7.9,2,Connecticut,14577.0,0.910380,0.672848,10774
Delaware,5555.0,989948,7.8,2,Delaware,5132.0,0.923852,0.672848,3738
District of Columbia,5672.0,689545,8.0,2,District of Columbia,4436.0,0.782087,0.672848,3816
Florida,81365.0,21538187,7.7,0,Florida,NaN,NaN,0.633908,51578


In [ ]:
# scatterplot y axis death covid, x axis opioid deaths 
# manipulate the data so that the data is all binned by month and state
# average of each state each month for covid set
# df should look like:
# State - Opioid Deaths per capita - COVID Deaths per capita - Month
def covidopioidplot2020(state):
    if state in stateswithavailabledata:
        temp1 = dfCOVID2020byMonth[(dfCOVID2020byMonth['state'] == state)]
        temp2 = dfOpioid2020byMonth[(dfOpioid2020byMonth['State Name'] == state)]
        temp1 = temp1.sort_values('date')
        temp2 = temp2.sort_values('Date')        
        x_data = temp1['date']
        temp1 = temp1.set_index('date')
        temp2 = temp2.set_index('Date')
        y_data1 = temp1['Monthly COVID Deaths 2020']
        y_data2 = temp2['Data Value']
        plt.plot(x_data, y_data1, label = "COVID")
        plt.plot(x_data, y_data2, label = "Opioid")    
        plt.xlabel("Date")
        plt.ylabel("Number of Deaths")
        plt.title("COVID vs Opioid Deaths 2020 for %s" % (state))       
        plt.legend()
        plt.show()
    else:
        temp1 = dfCOVID2020byMonth.copy()
        temp2 = dfOpioid2020byMonth.copy()
        temp1['totalCD'] = (dfCOVID2020byMonth.groupby([pd.Grouper(key='date', freq='1M')])['Monthly COVID Deaths 2020'].transform('sum'))
        temp2['totalOD'] = (dfOpioid2020byMonth.groupby([pd.Grouper(key='Date', freq='1M')])['Data Value'].transform('sum'))
        temp1 = temp1[['date', 'totalCD']]
        temp2 = temp2[['Date','totalOD']]
        temp1 = temp1.sort_values('date')
        temp2 = temp2.sort_values('Date')
        temp1 = temp1.drop_duplicates()
        temp2 = temp2.drop_duplicates()
        x_data = temp1['date']
        temp1 = temp1.set_index('date')
        temp2 = temp2.set_index('Date')
        y_data1 = temp1['totalCD']
        y_data2 = temp2['totalOD']
        plt.plot(x_data, y_data1, label = "COVID")
        plt.plot(x_data, y_data2, label = "Opioid")    
        plt.xlabel("Date")
        plt.ylabel("Number of Deaths")
        plt.title("COVID vs Opioid Deaths 2020 for %s" % ("All States with Available Data"))       
        plt.legend()
        plt.show()    

In [ ]:
# next step, doing respective plotting as required 

# measures of central tendency
descriptivestatistics(dfTypeDrug, 'Data Value')
descriptivestatistics(dfStateC19Death, 'deaths')

In [ ]:
# # time plot for deaths for opioid over time for some states
# for i, state in enumerate(stateswithavailabledata):
#     if i % 4 == 1:
#         covidopioidplot2020(state)
        
# time plot for deaths for opioid over time for all states' data combined
covidopioidplot2020("All")

# scatterplot opioid vs covid per capita by state
scatterplot(dfpercapita['OpioidDeathsPerCapita'], dfpercapita['C19DeathsPerCapita'], "2020 Opioid Deaths", "2020 COVID Deaths")

# opioid deaths over time
timeplot(df_18_20_Opioid['Date'], df_18_20_Opioid['Total Deaths per Month'], "Date", "Total Deaths per Month")


# xdata = dfCOVIDbyMonth.copy()
# dfCOVIDbyMonth = dfCOVIDbyMonth.set_index('date')
# timeplot(xdata['date'], dfCOVIDbyMonth['Total Monthly COVID Cases'], "Date", "Total Cases per Month")
# timeplot(xdata['date'], dfCOVIDbyMonth['Total Monthly COVID Deaths'], "Date", "Total Deaths per Month")

# # employement on scatter plot, one with opioid one with covid 
scatterplot(dfUnemployment2020["Unemployment_rate_2020"], dfOpioid2020['Total Opioid Deaths 2020'], "2020 Unemployment Rate", "Opioid Deaths")
scatterplot(dfUnemployment2020["Unemployment_rate_2020"], dfCOVID2020['Total COVID Deaths 2020'], "2020 Unemployment Rate", "COVID Deaths")
scatterplot(dfUnemployment2020["Unemployment_rate_2020"], dfCOVID2020['Total COVID Cases 2020'], "2020 Unemployment Rate", "COVID Cases")

# # race and unemployment connecting to opioid overdose deaths 
# scatterplot(x_data, y_data, "Race Employment", "Opioid Deaths")
# scatterplot(x_data, y_data, "Race Employment", "Opioid Deaths")
# scatterplot(x_data, y_data, "Race Employment", "Opioid Deaths")
# scatterplot(x_data, y_data, "Race Employment", "Opioid Deaths")
# scatterplot(x_data, y_data, "Race Employment", "Opioid Deaths")


# # race and unemployement connecting to covid deaths 
# scatterplot(x_data, y_data, "Race Employment", "COVID Deaths")
# scatterplot(x_data, y_data, "Race Employment", "COVID Deaths")
# scatterplot(x_data, y_data, "Race Employment", "COVID Deaths")
# scatterplot(x_data, y_data, "Race Employment", "COVID Deaths")
# scatterplot(x_data, y_data, "Race Employment", "COVID Deaths")
# scatterplot(x_data, y_data, "Race Employment", "COVID Deaths")

# arcgis mapping??

In [ ]:
# after plotting, we need to do regression analysis, cluster analysis and so forth

